## import modules

In [1]:
from tqdm import tqdm
import re
import torch
import os
import random
import json
import numpy as np
import gc

import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback
from transformers import AutoModelForSequenceClassification, AutoConfig, AutoTokenizer

In [2]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"  # 디버깅 위한 세팅
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

print('Device:', torch.cuda.device)  # 출력결과: cuda 
print('Count of using GPUs:', torch.cuda.device_count())   
print('Current cuda device:', torch.cuda.current_device()) 

Device: <class 'torch.cuda.device'>
Count of using GPUs: 2
Current cuda device: 0


### Load Data
Data가 위치한 PATH에서 Data를 불러오기

In [3]:
def jsonload(fname, encoding="utf-8"):
    with open(fname, encoding=encoding) as f:
        j = json.load(f)

    return j

# json 개체를 파일이름으로 깔끔하게 저장
def jsondump(j, fname):
    with open(fname, "w", encoding="UTF8") as f:
        json.dump(j, f, ensure_ascii=False)

# jsonl 파일 읽어서 list에 저장
def jsonlload(fname, encoding="utf-8"):
    json_list = []
    with open(fname, encoding=encoding) as f:
        for line in f.readlines():
            json_list.append(json.loads(line))
    return json_list

In [4]:
train_data = jsonlload('nikluge-sa-2022-train.jsonl')
dev_data = jsonlload('nikluge-sa-2022-dev.jsonl')

## Preprocessing method

In [5]:
def preprocess(text: str, only_kor: bool = True):
    """한국어 문장을 옵션에 맞게 전처리"""
    # 한국어 모음과 특수 문자, 숫자 및 영어 제거
    if only_kor:
        text = re.sub(f"[^가-힣| |]+", "", text)
    else:
        text = re.sub(f"[^가-힣|ㄱ-ㅎ|0-9|]+", "", text)

    # 연속 공백 제거
    text = re.sub(" +", " ", text)

    # 좌우 불필요한 공백 제거
    return text.strip()

## 재현성 위한 seed 고정

In [6]:
def seed_everything(seed:int = 1004):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
    
seed_everything(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## 태그셋 정의

In [7]:
#개체#속성 태그셋 정의
entity_property_pair = [
    '제품 전체#품질', '제품 전체#편의성', '제품 전체#디자인', '제품 전체#일반', '제품 전체#가격', 
    '제품 전체#인지도',  '제품 전체#다양성',
    '패키지/구성품#디자인', '패키지/구성품#가격', '패키지/구성품#다양성', '패키지/구성품#일반',
    '패키지/구성품#편의성', '패키지/구성품#품질',
    '본품#일반', '본품#다양성', '본품#품질', '본품#편의성', '본품#디자인', '본품#가격',
    '브랜드#인지도', '브랜드#일반', '브랜드#디자인', '브랜드#품질', '브랜드#가격']

rep_entity_property_pair = [
    '제품전체#품질', '제품전체#편의성', '제품전체#디자인', '제품전체#일반', '제품전체#가격', 
    '제품전체#인지도',
    '패키지/구성품#디자인', '패키지/구성품#일반',
    '패키지/구성품#편의성', '패키지/구성품#품질',
    '본품#일반', '본품#다양성', '본품#품질', '본품#편의성', '본품#디자인',
    '브랜드#인지도', '브랜드#일반', '브랜드#품질']


"""
Train데이터에 등장 안하거나 희소하게 등장한 태그들 ==> rep에서 모두 지워주었음.

제품 전체#다양성  0

패키지/구성품#가격 0 
패키지/구성품#다양성 1

본품#인지도 1
본품#가격  2
본품#인지도 1

브랜드#디자인  0
브랜드#가격  3
"""

len(rep_entity_property_pair)

18

## Load Tokenizer

In [8]:
MODEL_NAME = 'klue/roberta-large'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer

PreTrainedTokenizerFast(name_or_path='klue/roberta-large', vocab_size=32000, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [9]:
special_tokens_dict = {
    'additional_special_tokens': rep_entity_property_pair
}

print(tokenizer.vocab_size)

# 토크나이저는 각 모델 별로 만들지 않아도 된다. [임베딩 layer에만 영향 주므로]
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

print(num_added_toks)

32000
18


In [10]:
print(tokenizer.tokenize("야 아이패드 가격 왜 이래 본품#일반"))
print(tokenizer("야 아이패드 가격 왜 이래 본품#일반"))
print(tokenizer.decode(tokenizer.encode("야 아이패드 가격 왜 이래 본품#일반")))

['야', '아이패드', '가격', '왜', '이래', '본품#일반']
{'input_ids': [0, 1396, 15641, 3852, 1460, 5625, 32010, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}
[CLS] 야 아이패드 가격 왜 이래 본품#일반 [SEP]


## Load Model

In [11]:
"""ACD_set = []

for i in range(len(rep_entity_property_pair)):
    tmp_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)  #True/False 
    # tmp_model.resize_token_embeddings(tokenizer.vocab_size + num_added_toks)
    # ACD 모델은 태그셋이 input으로 들어가지 않으므로 resize 안함.
    ACD_set.append(tmp_model)

ACD_set[7]"""

'ACD_set = []\n\nfor i in range(len(rep_entity_property_pair)):\n    tmp_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)  #True/False \n    # tmp_model.resize_token_embeddings(tokenizer.vocab_size + num_added_toks)\n    # ACD 모델은 태그셋이 input으로 들어가지 않으므로 resize 안함.\n    ACD_set.append(tmp_model)\n\nACD_set[7]'

## 데이터 준비

In [12]:
polarity_id_to_name = ['positive', 'negative', 'neutral']  # 차례대로 0, 1, 2
polarity_name_to_id = {polarity_id_to_name[i]: i for i in range(len(polarity_id_to_name))}

In [13]:
ACD_Datas = [[] for i in range(len(entity_property_pair))]  # 속성 각각마다의 데이터 셋
ACD_labels = [[] for i in range(len(entity_property_pair))]

POL_Datas = []
POL_labels = []

### sentence들 먼저 모으고 한꺼번에 tokenize 하자

for idx, pair in enumerate(rep_entity_property_pair):
    for idx2, datas in enumerate(train_data):
        sen = preprocess(datas['sentence_form'])
        annos = datas['annotation']  # 여러 개 일 수도 있음. (이중 리스트)
        check_point = False

        """
        개체#속성 태그 셋의 경우 sentence를 보고 모델이 태그를 True/False로 알려줄 것이므로 pair를 넣으면 안됨.
        
        감성 분석을 위한 데이터셋의 경우 sentence + 태그를 본 후 해당 태그에 대한 감성 분석을 기대하므로 pair를 함께 넣어줌.
        """
        
        ACD_Datas[idx].append(sen)
        
        for idx3, annotation in enumerate(annos):
            entity_property = annotation[0]  # raw_data의 annotation 추출
            entity_property = entity_property.replace(" ", "")  # 띄어쓰기 제거
            polarity = annotation[2]

            if entity_property == pair:
                check_point = True
                
            else:
                ;
                
        if check_point:
            ACD_labels[idx].append(1)  # 해당 태그 맞음.
            POL_Datas.append(sen + " "+ pair)
            POL_labels.append(polarity_name_to_id[polarity])
        
        else:
            ACD_labels[idx].append(0)
            
    ACD_Datas[idx], ACD_labels[idx] = shuffle(ACD_Datas[idx], ACD_labels[idx], random_state = 42)
    
POL_Datas, POL_labels = shuffle(POL_Datas, POL_labels, random_state = 42)

In [14]:
devs = dev_data[:838]
tests = dev_data[838:]

In [15]:
################### Dev set #######################
dev_ACD_Datas = [[] for i in range(len(entity_property_pair))]  # 속성 각각마다의 데이터 셋
dev_ACD_labels = [[] for i in range(len(entity_property_pair))]

dev_POL_Datas = []
dev_POL_labels = []

### sentence들 먼저 모으고 한꺼번에 tokenize 하자

for idx, pair in enumerate(rep_entity_property_pair):
    for idx2, datas in enumerate(devs):
        sen = preprocess(datas['sentence_form'])
        annos = datas['annotation']  # 여러 개 일 수도 있음. (이중 리스트)
        check_point = False

        """
        개체#속성 태그 셋의 경우 sentence를 보고 모델이 태그를 True/False로 알려줄 것이므로 pair를 넣으면 안됨.
        
        감성 분석을 위한 데이터셋의 경우 sentence + 태그를 본 후 해당 태그에 대한 감성 분석을 기대하므로 pair를 함께 넣어줌.
        """
        
        dev_ACD_Datas[idx].append(sen)
        
        for idx3, annotation in enumerate(annos):
            entity_property = annotation[0]  # raw_data의 annotation 추출
            entity_property = entity_property.replace(" ", "")  # 띄어쓰기 제거
            polarity = annotation[2]

            if entity_property == pair:
                check_point = True
                
            else:
                ;
                
        if check_point:
            dev_ACD_labels[idx].append(1)  # 해당 태그 맞음.
            dev_POL_Datas.append(sen + " "+ pair)
            dev_POL_labels.append(polarity_name_to_id[polarity])
        
        else:
            dev_ACD_labels[idx].append(0)
        
    dev_ACD_Datas[idx], dev_ACD_labels[idx] = shuffle(dev_ACD_Datas[idx], dev_ACD_labels[idx], random_state = 42)
    
dev_POL_Datas, dev_POL_labels = shuffle(dev_POL_Datas, dev_POL_labels, random_state = 42)

In [16]:
################### Test set #######################
test_ACD_Datas = [[] for i in range(len(entity_property_pair))]  # 속성 각각마다의 데이터 셋
test_ACD_labels = [[] for i in range(len(entity_property_pair))]

test_POL_Datas = []
test_POL_labels = []

### sentence들 먼저 모으고 한꺼번에 tokenize 하자

for idx, pair in enumerate(rep_entity_property_pair):
    for idx2, datas in enumerate(tests):
        sen = preprocess(datas['sentence_form'])
        annos = datas['annotation']  # 여러 개 일 수도 있음. (이중 리스트)
        check_point = False

        """
        개체#속성 태그 셋의 경우 sentence를 보고 모델이 태그를 True/False로 알려줄 것이므로 pair를 넣으면 안됨.
        
        감성 분석을 위한 데이터셋의 경우 sentence + 태그를 본 후 해당 태그에 대한 감성 분석을 기대하므로 pair를 함께 넣어줌.
        """
        
        test_ACD_Datas[idx].append(sen)
        
        for idx3, annotation in enumerate(annos):
            entity_property = annotation[0]  # raw_data의 annotation 추출
            entity_property = entity_property.replace(" ", "")  # 띄어쓰기 제거
            polarity = annotation[2]

            if entity_property == pair:
                check_point = True
                
            else:
                ;
                
        if check_point:
            test_ACD_labels[idx].append(1)  # 해당 태그 맞음.
            test_POL_Datas.append(sen + " "+ pair)
            test_POL_labels.append(polarity_name_to_id[polarity])
        
        else:
            test_ACD_labels[idx].append(0)
            
    test_ACD_Datas[idx], test_ACD_labels[idx] = shuffle(test_ACD_Datas[idx], test_ACD_labels[idx], random_state = 42)
    
test_POL_Datas, test_POL_labels = shuffle(test_POL_Datas, test_POL_labels, random_state = 42)

In [17]:
class klue_Dataset(torch.utils.data.Dataset):
    def __init__(self, dataset, label):  # 전처리된 데이터 셋이 들어옴
        self.dataset = dataset
        self.label = label

    def __getitem__(self, idx):
        # getitem이므로 gradient 계산에 영향을 주지 않게 clone().detach() 실행
        item = {key: val[idx].clone().detach() for key, val in self.dataset.items()}
        item['label'] = torch.tensor(self.label[idx])
        
        return item

    def __len__(self):  # 샘플 수
        return len(self.label)

In [18]:
klue_sets = []

for i in range(len(rep_entity_property_pair)):
    tok_sen = tokenizer(ACD_Datas[i], padding='max_length', return_tensors="pt",
                    max_length=256, truncation=True, add_special_tokens=True)  
    
    klue_sets.append(klue_Dataset(tok_sen, ACD_labels[i]))  # klue_Dataset에서 1차원 텐서로 바뀜.

In [19]:
klue_sets[5].__getitem__(758)

{'input_ids': tensor([    0, 28111, 25535,  8158,  2200,  5404,  3806,  7704,  2318, 13928,
         18713,     2,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

In [20]:
dev_klue_sets = []

for i in range(len(rep_entity_property_pair)):
    tok_sen = tokenizer(dev_ACD_Datas[i], padding='max_length', return_tensors="pt",
                    max_length=256, truncation=True, add_special_tokens=True)  
    
    dev_klue_sets.append(klue_Dataset(tok_sen, dev_ACD_labels[i]))  # klue_Dataset에서 1차원 텐서로 바뀜.
    
dev_klue_sets[5].__getitem__(758)

{'input_ids': tensor([    0, 11962,  2122,  5077, 27135,   677,  1233,  2275,  2205,  2259,
          9648,  2643, 15116,  2643,  5576,  2119,  2205,  2321,     2,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

In [21]:
test_klue_sets = []

for i in range(len(rep_entity_property_pair)):
    tok_sen = tokenizer(test_ACD_Datas[i], padding='max_length', return_tensors="pt",
                    max_length=256, truncation=True, add_special_tokens=True)  
    
    test_klue_sets.append(klue_Dataset(tok_sen, test_ACD_labels[i]))  # klue_Dataset에서 1차원 텐서로 바뀜.
    
test_klue_sets[5].__getitem__(758)

{'input_ids': tensor([    0, 21253,  2170,  2318,  2457,  2608,  2073,  2311,  2323,  2047,
             2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

### Polarity datasets

In [22]:
pol_tok_sen = tokenizer(POL_Datas, padding='max_length', return_tensors="pt",
                    max_length=256, truncation=True, add_special_tokens=True)  

POL_klue_sets = klue_Dataset(pol_tok_sen, POL_labels)

POL_klue_sets.__getitem__(758)

{'input_ids': tensor([    0,  4482,  2119,  7693,  2088, 11410,  2073,   543,   554,  2088,
           769,  2015,  2119,  1889,  2259,  2284, 32006,     2,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

In [23]:
pol_tok_sen = tokenizer(dev_POL_Datas, padding='max_length', return_tensors="pt",
                    max_length=256, truncation=True, add_special_tokens=True)  

dev_POL_klue_sets = klue_Dataset(pol_tok_sen, dev_POL_labels)

dev_POL_klue_sets.__getitem__(758)

{'input_ids': tensor([    0,   831,  2057,  4039,  2052,  7136,  3940, 21820,  1111,  2089,
          2094,  2200,  2197, 32012,     2,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

In [24]:
pol_tok_sen = tokenizer(test_POL_Datas, padding='max_length', return_tensors="pt",
                    max_length=256, truncation=True, add_special_tokens=True)  

test_POL_klue_sets = klue_Dataset(pol_tok_sen, test_POL_labels)

test_POL_klue_sets.__getitem__(758)

{'input_ids': tensor([    0, 11174,  6396,  9989,  2585,  2051,  9497,  2683,  9151, 13937,
         30669,   991,  9534,  2612,  2613,  4212,  2223,  2015, 32003,     2,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

## Trainer 정의

In [25]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
    }

In [52]:
for i in range(len(rep_entity_property_pair)):
    if i<14 or i>14:
        continue;
    MODEL_NAME = 'klue/roberta-large'
    tmp_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

    odir = '/mnt/HDD4T/egg2018037024/ABSA' + str(i)

    training_ars = TrainingArguments(
    output_dir=odir,
    num_train_epochs=20,
    #    max_steps=5000,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    save_total_limit=5,
    save_strategy = "epoch",
    #    save_steps=1000,
    learning_rate=1e-6,
    weight_decay=0.01,
    #    eval_steps=1000,
    evaluation_strategy='epoch',
    metric_for_best_model = 'f1',
    load_best_model_at_end = True,
    )

    trainer = Trainer(
    model=tmp_model,
    args=training_ars,
    train_dataset=klue_sets[i],
    eval_dataset=dev_klue_sets[i],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=4)]
    )

    print(i, "th Trarining.... ========================================")
    trainer.train()
    tmp_model.save_pretrained(odir + "_best")

    with torch.no_grad(): tmp_model
    gc.collect()
    torch.cuda.empty_cache()

loading configuration file https://huggingface.co/klue/roberta-large/resolve/main/config.json from cache at /home/egg2018037024/.cache/huggingface/transformers/571e05a2160c18c93365862223c4dae92bbd1b41464a4bd5f372ad703dba6097.ae5b7f8d8a28a3ff0b1560b4d08c6c3bd80f627288eee2024e02959dd60380d0
Model config RobertaConfig {
  "_name_or_path": "klue/roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertTokenizer",
  "transformers_version": "4.16.2",
  "type_vocab_size": 1,
  "u

14 th Trarining.... ========================================


/home/egg2018037024/venv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.026805,0.996420,0.000000,0.000000,0.000000
2,0.074100,0.027728,0.996420,0.000000,0.000000,0.000000
3,0.037000,0.028548,0.996420,0.000000,0.000000,0.000000
4,0.029300,0.027997,0.996420,0.000000,0.000000,0.000000
5,0.029300,0.031160,0.996420,0.000000,0.000000,0.000000


***** Running Evaluation *****
  Num examples = 838
  Batch size = 8
/home/egg2018037024/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to /mnt/HDD4T/egg2018037024/ABSA14/checkpoint-376
Configuration saved in /mnt/HDD4T/egg2018037024/ABSA14/checkpoint-376/config.json
Model weights saved in /mnt/HDD4T/egg2018037024/ABSA14/checkpoint-376/pytorch_model.bin
tokenizer config file saved in /mnt/HDD4T/egg2018037024/ABSA14/checkpoint-376/tokenizer_config.json
Special tokens file saved in /mnt/HDD4T/egg2018037024/ABSA14/checkpoint-376/special_tokens_map.json
/home/egg2018037024/venv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were sc

## Test set 평가

In [57]:
infers = [[] for i in range(len(rep_entity_property_pair))]
infer_labels = [[] for i in range(len(rep_entity_property_pair))]

for i in range(len(rep_entity_property_pair)):
    print(i, "th Test.... ========================================")
    MODEL_NAME = '/mnt/HDD4T/egg2018037024/ABSA' + str(i) + "_best"
    
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
    model.to(device)
    dataloader = DataLoader(test_klue_sets[i], batch_size=4, shuffle=False)

    model.eval()
    output_pred = []
    output_prob = []
    labels = []

    for z, data in enumerate(tqdm(dataloader)):
        with torch.no_grad():
            outputs = model(
                input_ids=data['input_ids'].to(device),
                attention_mask=data['attention_mask'].to(device),
                token_type_ids=data['token_type_ids'].to(device)
            )
        logits = outputs[0]
        prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
        logits = logits.detach().cpu().numpy()
        result = np.argmax(logits, axis=-1)
        labels.append(data['label'].tolist())

        output_pred.append(result)
        output_prob.append(prob)

    pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()
    
    infers[i].extend(pred_answer)
    infer_labels[i].extend(labels)
    #   print(pred_answer)
  #  print(output_prob)

loading configuration file /mnt/HDD4T/egg2018037024/ABSA0_best/config.json
Model config RobertaConfig {
  "_name_or_path": "/mnt/HDD4T/egg2018037024/ABSA0_best",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "tokenizer_class": "BertTokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 32000
}

loading weights file /mnt/HDD4T/egg2018037024/ABSA0_best/pytorch_

0 th Test.... ========================================


All model checkpoint weights were used when initializing RobertaForSequenceClassification.

All the weights of RobertaForSequenceClassification were initialized from the model checkpoint at /mnt/HDD4T/egg2018037024/ABSA0_best.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RobertaForSequenceClassification for predictions without further training.
100%|█████████████████████████████████████████| 489/489 [00:15<00:00, 30.74it/s]
loading configuration file /mnt/HDD4T/egg2018037024/ABSA1_best/config.json
Model config RobertaConfig {
  "_name_or_path": "/mnt/HDD4T/egg2018037024/ABSA1_best",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_

1 th Test.... ========================================


All model checkpoint weights were used when initializing RobertaForSequenceClassification.

All the weights of RobertaForSequenceClassification were initialized from the model checkpoint at /mnt/HDD4T/egg2018037024/ABSA1_best.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RobertaForSequenceClassification for predictions without further training.
100%|█████████████████████████████████████████| 489/489 [00:15<00:00, 30.70it/s]
loading configuration file /mnt/HDD4T/egg2018037024/ABSA2_best/config.json
Model config RobertaConfig {
  "_name_or_path": "/mnt/HDD4T/egg2018037024/ABSA2_best",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_

...

All model checkpoint weights were used when initializing RobertaForSequenceClassification.

All the weights of RobertaForSequenceClassification were initialized from the model checkpoint at /mnt/HDD4T/egg2018037024/ABSA15_best.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RobertaForSequenceClassification for predictions without further training.
100%|█████████████████████████████████████████| 489/489 [00:15<00:00, 30.59it/s]
loading configuration file /mnt/HDD4T/egg2018037024/ABSA16_best/config.json
Model config RobertaConfig {
  "_name_or_path": "/mnt/HDD4T/egg2018037024/ABSA16_best",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "lay

16 th Test.... ========================================


All model checkpoint weights were used when initializing RobertaForSequenceClassification.

All the weights of RobertaForSequenceClassification were initialized from the model checkpoint at /mnt/HDD4T/egg2018037024/ABSA16_best.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RobertaForSequenceClassification for predictions without further training.
100%|█████████████████████████████████████████| 489/489 [00:15<00:00, 30.58it/s]
loading configuration file /mnt/HDD4T/egg2018037024/ABSA17_best/config.json
Model config RobertaConfig {
  "_name_or_path": "/mnt/HDD4T/egg2018037024/ABSA17_best",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "lay

17 th Test.... ========================================


All model checkpoint weights were used when initializing RobertaForSequenceClassification.

All the weights of RobertaForSequenceClassification were initialized from the model checkpoint at /mnt/HDD4T/egg2018037024/ABSA17_best.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RobertaForSequenceClassification for predictions without further training.
100%|█████████████████████████████████████████| 489/489 [00:16<00:00, 30.41it/s]


In [60]:
for x in range(len(rep_entity_property_pair)):
    print(x, "th Test.... ========================================")
    labelss = []

    for i in infer_labels[x]:
        for j in i:
            labelss.append(j)
        
    print(len(labelss), len(infers[x]))
    precision, recall, f1, _ = precision_recall_fscore_support(labelss, infers[x], average='binary')
    acc = accuracy_score(labelss, infers[x])
    
    print("Accuracy: ", acc)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-score:", f1)

0 th Test.... ========================================
1956 1956
Accuracy:  0.9008179959100204
Precision: 0.43089430894308944
Recall: 0.2994350282485876
F1-score: 0.3533333333333334
1 th Test.... ========================================
1956 1956
Accuracy:  0.9570552147239264
Precision: 0.3763440860215054
Recall: 0.5737704918032787
F1-score: 0.4545454545454546
2 th Test.... ========================================
1956 1956
Accuracy:  0.9662576687116564
Precision: 0.632183908045977
Recall: 0.6179775280898876
F1-score: 0.625
3 th Test.... ========================================
1956 1956
Accuracy:  0.8629856850715747
Precision: 0.7562380038387716
Recall: 0.7364485981308411
F1-score: 0.7462121212121211
4 th Test.... ========================================
1956 1956
Accuracy:  0.9969325153374233
Precision: 0.8918918918918919
Recall: 0.9428571428571428
F1-score: 0.9166666666666667
5 th Test.... ========================================
1956 1956
Accuracy:  0.9815950920245399
Precision: 0.

/home/egg2018037024/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/egg2018037024/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/egg2018037024/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/egg2018037024/venv/lib/python3.8/site-packa

## Polarity model 학습

In [26]:
MODEL_NAME = 'klue/roberta-large'
tmp_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3)
tmp_model.resize_token_embeddings(tokenizer.vocab_size + num_added_toks)

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'class

Embedding(32018, 1024)

In [28]:
def pol_compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
    }

In [27]:
odir = '/mnt/HDD4T/egg2018037024/ABSA_pol'

training_ars = TrainingArguments(
output_dir=odir,
num_train_epochs=20,
#    max_steps=5000,
per_device_train_batch_size=4,
per_device_eval_batch_size=4,
save_total_limit=5,
save_strategy = "epoch",
#    save_steps=1000,
learning_rate=1e-6,
weight_decay=0.01,
#    eval_steps=1000,
evaluation_strategy='epoch',
metric_for_best_model = 'f1',
load_best_model_at_end = True,
)

trainer = Trainer(
model=tmp_model,
args=training_ars,
train_dataset=POL_klue_sets,
eval_dataset=dev_POL_klue_sets,
tokenizer=tokenizer,
compute_metrics=compute_metrics,
callbacks = [EarlyStoppingCallback(early_stopping_patience=4)]
)

print(i, "th Trarining.... ========================================")
trainer.train()
tmp_model.save_pretrained(odir + "_best")

/home/egg2018037024/venv/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3190
  Num Epochs = 20
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7980


17 th Trarining.... ========================================


/home/egg2018037024/venv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 913
  Batch size = 8


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].